# Graph Convolutional Network by Kipf and Welling

## Imports

In [3]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

import pickle
import numpy as np

import itertools

import Notebooks.performance as pf

## GNN Definition

In [4]:
class LinearModule(nn.Module):
    """The linear transformation part of the GCN layer"""
    def __init__(self, in_feats, out_feats, activation):
        super(LinearModule, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)
        self.activation = activation # This is the activation function

    def forward(self, node):
        h = self.linear(node.data['h'])
        h = self.activation(h)
        return {'h' : h}

In [5]:
class GCN(nn.Module):
    """A GCN layer"""
    def __init__(self, in_feats, out_feats, activation):
        super(GCN, self).__init__()
        self.apply_mod = LinearModule(in_feats, out_feats, activation)

    def forward(self, g, feature):
        g.ndata['h'] = feature
        g.update_all(message_func=fn.copy_src(src='h', out='m'), reduce_func=fn.sum(msg='m', out='h'))
        g.apply_nodes(func=self.apply_mod)
        return g.ndata.pop('h')

In [6]:
class Net(nn.Module):
    def __init__(self, infeats, hidden_size, outfeats):
        super(Net, self).__init__()
        self.gcn1 = GCN(infeats, hidden_size, F.relu)
        self.gcn2 = GCN(hidden_size, outfeats, lambda x: F.log_softmax(x,1))
        self.dropout = nn.Dropout(0.2)

    def forward(self, g, features):
        x = self.gcn1(g, features)
        #x = self.dropout(x)
        x = self.gcn2(g, x)
        return x

## Data Loading

In [7]:
from dgl.data import citation_graph as citegrh
import networkx as nx

data = citegrh.load_cora()
features = th.FloatTensor(data.features)
labels = th.LongTensor(data.labels)
mask = th.ByteTensor(data.train_mask)
g = data.graph

# add self loop
g.remove_edges_from(nx.selfloop_edges(g))
g = DGLGraph(g)
g.add_edges(g.nodes(), g.nodes())

## Select Training Set

In [8]:
percentage_train = 0.02

with open("data/cora_permutation1.pickle","rb") as f:
    perm1 = pickle.load(f)
mask = np.zeros(g.number_of_nodes())
mask[perm1[range(int(percentage_train*g.number_of_nodes()))]] = 1
mask = th.ByteTensor(mask)

## Training

In [9]:
loss_function = pf.perm_inv_loss(labels)

In [24]:
import time

net = Net(features.shape[1], 21, len(np.unique(labels)))
#print(net)

optimizer = th.optim.Adam(net.parameters(), lr=1e-3)
net.train() # Set to training mode (use dropout)

dur = []
for epoch in range(500):
    if epoch >=3:
        t0 = time.time()

    # Compute loss for test nodes (only for validation, not used by optimizer)
    net.eval()
    prediction = net(g, features)
    train_rand=pf.rand_score(labels[mask].numpy(),np.argmax(prediction[mask].detach().numpy(), axis=1)
    validation_rand=pf.rand_score(labels[1-mask].numpy(),np.argmax(prediction[1-mask].detach().numpy(), axis=1))
    net.train()

    # Compute loss for train nodes
    logits = net(g, features)

    loss = loss_function.approximate_loss(logits,mask,nclasses=5)

    #loss = F.nll_loss(logits[mask], labels[mask])
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >=3:
        dur.append(time.time() - t0)

    print("Epoch {:05d} | Loss {:.4f} | Valid.Rand {:.4f} | Time(s) {:.4f}".format(
            epoch, loss.item(), validation, np.mean(dur)))

Rand 0.1258 | Time(s) 0.2378
Epoch 00174 | Loss 1.9274 | Valid.Rand 0.1275 | Time(s) 0.2377
Epoch 00175 | Loss 1.9972 | Valid.Rand 0.1260 | Time(s) 0.2379
Epoch 00176 | Loss 1.8619 | Valid.Rand 0.1265 | Time(s) 0.2377
Epoch 00177 | Loss 1.8887 | Valid.Rand 0.1266 | Time(s) 0.2375
Epoch 00178 | Loss 2.1493 | Valid.Rand 0.1265 | Time(s) 0.2379
Epoch 00179 | Loss 2.0027 | Valid.Rand 0.1258 | Time(s) 0.2377
Epoch 00180 | Loss 1.9271 | Valid.Rand 0.1267 | Time(s) 0.2375
Epoch 00181 | Loss 2.3715 | Valid.Rand 0.1275 | Time(s) 0.2377
Epoch 00182 | Loss 1.9558 | Valid.Rand 0.1299 | Time(s) 0.2377
Epoch 00183 | Loss 2.3489 | Valid.Rand 0.1297 | Time(s) 0.2375
Epoch 00184 | Loss 2.1020 | Valid.Rand 0.1288 | Time(s) 0.2374
Epoch 00185 | Loss 1.8529 | Valid.Rand 0.1274 | Time(s) 0.2375
Epoch 00186 | Loss 2.0363 | Valid.Rand 0.1241 | Time(s) 0.2374
Epoch 00187 | Loss 1.9734 | Valid.Rand 0.1247 | Time(s) 0.2373
Epoch 00188 | Loss 2.1425 | Valid.Rand 0.1264 | Time(s) 0.2376
Epoch 00189 | Loss 2.1634 

In [20]:
# Visualise predictions
net.eval() # Set net to evaluation mode (deactivates dropout)
final_prediction = net(g, features).detach()
a = np.transpose(np.vstack([final_prediction[mask].numpy().argmax(axis=1),labels[mask].numpy()]))
a[a[:,0].argsort()][np.random.choice(range(a.shape[0]),size=10)]
# as can be seen, the net predicts other labels, but gets the clusters right :)

array([[0, 6],
       [0, 3],
       [2, 2],
       [2, 2],
       [0, 6],
       [6, 4],
       [4, 0],
       [4, 0],
       [0, 2],
       [0, 4]], dtype=int64)

## Evaluation

In [21]:
net.eval() # Set net to evaluation mode (deactivates dropout)
final_prediction = net(g, features).detach()
pf.performance_as_df(labels,final_prediction,mask)

,All,Train,Test
Mutual Information,0.200959,0.668763,0.193395
Rand-Index,0.092219,0.456485,0.088449
Variation of Information,2.655766,0.936472,2.667611
